In [1]:
################### 데이터 불러오기
import numpy
import pandas as pd
# import ga

"""
The y=target is to maximize this equation ASAP:
    y = w1x1+w2x2+w3x3+w4x4+w5x5+6wx6
    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7)
    What are the best values for the 6 weights w1 to w6?
    We are going to use the genetic algorithm for the best possible values after a number of generations.
"""
# data
restaurants = pd.read_csv("https://raw.githubusercontent.com/xoyeon/PDPTW/main/grubhub/0o100t100s1p100/restaurants.txt", sep = '\t')
orders = pd.read_csv("https://raw.githubusercontent.com/xoyeon/PDPTW/main/grubhub/0o100t100s1p100/orders.txt", sep = '\t')
couriers = pd.read_csv("https://raw.githubusercontent.com/xoyeon/PDPTW/main/grubhub/0o100t100s1p100/couriers.txt", sep = '\t')
rest_orders = pd.merge(restaurants, orders, how='inner', on=['restaurant'])
rest_orders.columns = ['restaurant', 'rx', 'ry', 'order', 'ox', 'oy', 'placement_time', 'ready_time']

# 주문 건
# order = rest_orders.sample(n=1)
order = ['r1',8708,5633,'o1',9131,7497,743,753]
print("order", order)
# 근무중인 라이더
rider = couriers[(couriers['on_time'] <= order[6]) & (couriers['off_time'] >= order[7])]
data_len = len(rider)
print("근무중인 라이더", len(rider), "명")

order ['r1', 8708, 5633, 'o1', 9131, 7497, 743, 753]
근무중인 라이더 9 명


In [2]:
"""
Genetic algorithm parameters:
    Mating pool size
    Population size
"""
# Defining the population size.
num_generations = 100

sol_per_pop = 50
num_parents_mating = 5
# weights = [0.1,0.3,0.5,0.7]

# Inputs of the equation. # 내가 업데이트 해야하는 값
numpy.random.seed(0)
delivered_num = numpy.random.uniform(low=3, high=25, size=data_len).astype(int)
dist_x = numpy.array(rider['x'])
dist_y = numpy.array(rider['y'])
on_time = numpy.array(rider['on_time'])
off_time = numpy.array(rider['off_time'])
speed = numpy.random.uniform(low=30, high=60, size=data_len).astype(int)
# trustiness = numpy.full(data_len, 0.90)

new_population = numpy.stack((delivered_num, dist_x, dist_y, on_time, off_time, speed), axis=1)
print("new_population",new_population)

# Number of the weights we are looking to optimize.
num_weights = 6

# The population will have sol_per_pop chromosome where each chromosome has num_weights genes.
pop_size = (sol_per_pop,num_weights)
print("pop_size",pop_size)

new_population [[   15  3810  7415   690   810    41]
 [   18  3217  9891   690   840    53]
 [   16 11678  3104   690   810    45]
 [   14  7651  4332   690   810    47]
 [   12 10855  7784   690   810    57]
 [   17  7038  5694   690   840    32]
 [   12  5013  2778   690   810    32]
 [   22  8236  1518   690   810    30]
 [   24  4981  4827   690   840    54]]
pop_size (50, 6)


mean = sum(new_population[:,0]) / len(new_population)
print("mean",mean)
var = (new_population[:,0]-25)**2 / len(new_population)
print("var",var)
std = var ** 0.5
print("std",std)

speed = new_population[:,5]
print("speed",speed)

trust = new_population[:,4]-new_population[:,3] / new_population[:,0]
print("trust",trust)

log_std = numpy.log1p(std)
print("log_std",log_std)
log_speed = numpy.log1p(speed)
print("log_speed",log_speed)
log_trust = numpy.log1p(trust)
print("log_trust",log_trust)


print("Fitness", 1/log_std + log_speed +log_trust)

In [3]:
new_population.shape

(9, 6)

In [4]:
import numpy

def standard_deviation(pop):
    mean = sum(pop[:,0]) / len(pop)
    var = (pop[:,0]-mean)**2 / len(pop)
    std = var ** 0.5
    return std

def calculate_speed(pop):
    speed = pop[:,5]
    return speed

def calculate_trust(pop):
    # 배달원의 총 배달 시간 / 배달 건 수
    trust = pop[:,4]-pop[:,3] / pop[:,0]
    return trust
    
# Calculating the fitness value of each solution in the current population.
# The fitness function calulates the sum of products between each input and its corresponding weight.

def cal_pop_fitness(pop):
    std = standard_deviation(pop)
    speed = calculate_speed(pop)
    trust = calculate_trust(pop)
    
    log_std = numpy.log1p(std)
    log_speed = numpy.log1p(speed)
    log_trust = numpy.log1p(trust)
    return 1/log_std + log_speed +log_trust

def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = numpy.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
    return parents

def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually, it is at the center.
    crossover_point = numpy.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

def mutation(offspring_crossover, num_mutations=1):
    mutations_counter = numpy.uint8(offspring_crossover.shape[1] / num_mutations)
    # Mutation changes a number of genes as defined by the num_mutations argument. The changes are random.
    for idx in range(offspring_crossover.shape[0]):
        gene_idx = mutations_counter - 1
        for mutation_num in range(num_mutations):
            # The random value to be added to the gene.
            random_value = numpy.random.uniform(-1.0, 1.0, 1)
            offspring_crossover[idx, gene_idx] = offspring_crossover[idx, gene_idx] + random_value
            gene_idx = gene_idx + mutations_counter
    return offspring_crossover

In [5]:
best_outputs = []
# num_generations = 500
for generation in range(num_generations):
    print("Generation : ", generation)
    # Measuring the fitness of each chromosome in the population.
    fitness = cal_pop_fitness(new_population)
    print("Fitness")
    print(fitness)

    best_outputs.append(numpy.max(fitness))
    # The best result in the current iteration.
    print("Best result : ", numpy.max(fitness))
    
    # Selecting the best parents in the population for mating.
    parents = select_mating_pool(new_population, fitness, 
                                      num_parents_mating)
    print("Parents")
    print(parents)

    # Generating next generation using crossover.
    offspring_crossover = crossover(parents, offspring_size=(sol_per_pop-parents.shape[0], num_weights))
                                    # offspring_size=(pop_size[0]-parents.shape[0], num_weights)) 
    print("Crossover")
    print(offspring_crossover)

    # Adding some variations to the offspring using mutation.
    offspring_mutation = mutation(offspring_crossover, num_mutations=2)
    print("Mutation")
    print(offspring_mutation)

    # Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation
    
# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
fitness = cal_pop_fitness(new_population)
# Then return the index of that solution corresponding to the best fitness.
best_match_idx = numpy.where(fitness == numpy.max(fitness))

print("Best solution : ", new_population[best_match_idx, :])
print("Best solution fitness : ", fitness[best_match_idx])


import matplotlib.pyplot
matplotlib.pyplot.plot(best_outputs)
matplotlib.pyplot.xlabel("Iteration")
matplotlib.pyplot.ylabel("Fitness")
matplotlib.pyplot.show()

Generation :  0
Fitness
[12.6408453  13.39634867 15.45555701 12.07912736 11.75096374 19.67285544
 11.18702829 11.07162241 11.51570396]
Best result :  19.672855443631057
Parents
[[   17.  7038.  5694.   690.   840.    32.]
 [   16. 11678.  3104.   690.   810.    45.]
 [   18.  3217.  9891.   690.   840.    53.]
 [   15.  3810.  7415.   690.   810.    41.]
 [   14.  7651.  4332.   690.   810.    47.]]
Crossover
[[   17.  7038.  5694.   690.   810.    45.]
 [   16. 11678.  3104.   690.   840.    53.]
 [   18.  3217.  9891.   690.   810.    41.]
 [   15.  3810.  7415.   690.   810.    47.]
 [   14.  7651.  4332.   690.   840.    32.]
 [   17.  7038.  5694.   690.   810.    45.]
 [   16. 11678.  3104.   690.   840.    53.]
 [   18.  3217.  9891.   690.   810.    41.]
 [   15.  3810.  7415.   690.   810.    47.]
 [   14.  7651.  4332.   690.   840.    32.]
 [   17.  7038.  5694.   690.   810.    45.]
 [   16. 11678.  3104.   690.   840.    53.]
 [   18.  3217.  9891.   690.   810.    41.]
 [

ValueError: could not broadcast input array from shape (45,6) into shape (4,6)